# Identify top decile of momentum stocks
> To calculate the cumulative return over the past 12 months, we take the net return streams from each month and turn them into gross returns by adding 1. Thus, if Apple's net returns for January are –10.77 percent, Apple's gross returns for January are 0.8923 (–0.1077 + 1). Then, we multiply all the gross return series (i.e., months) and subtract 1 to find the cumulative 12-month net return. ([Location 2467](https://readwise.io/to_kindle?action=open&asin=B01LY6P2LB&location=2467))

In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
import pandas_datareader as pdr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import arrow

In [ ]:
now = arrow.get('2017-10-03')
# calculate lookback period for momentum
# start period defined as the first day of the month, 13 months ago, enabling us to get initial value to compare
start_period = now.shift(months=-12).replace(day=1).format('YYYY-MM-DD')
# end period excludes the most recent full month, (exclusive)
end_period = now.shift(months=-1).replace(day=1).format('YYYY-MM-DD')
print(start_period)
print(end_period)
type(now)

In [ ]:
ticker = pdr.quandl.QuandlReader('AAPL', start_period, end_period).read()

In [ ]:
ticker = ticker.sort_index()

In [ ]:
candles = [{'open': 72.4825, 'high': 73.42, 'low': 72.38, 'close': 73.4125, 'volume': 100990500, 'datetime': 1577772000000}, {'open': 74.06, 'high': 75.15, 'low': 73.7975, 'close': 75.0875, 'volume': 135647456, 'datetime': 1577944800000}, {'open': 74.2875, 'high': 75.145, 'low': 74.125, 'close': 74.3575, 'volume': 146535512, 'datetime': 1578031200000}, {'open': 73.4475, 'high': 74.99, 'low': 73.1875, 'close': 74.95, 'volume': 118578576, 'datetime': 1578290400000}, {'open': 74.96, 'high': 75.225, 'low': 74.37, 'close': 74.5975, 'volume': 111510620, 'datetime': 1578376800000}, {'open': 74.29, 'high': 76.109975, 'low': 74.289, 'close': 75.7975, 'volume': 132363784, 'datetime': 1578463200000}, {'open': 76.80875, 'high': 77.6075, 'low': 76.55, 'close': 77.4075, 'volume': 170486168, 'datetime': 1578549600000}, {'open': 77.65, 'high': 78.1675, 'low': 77.0625, 'close': 77.5825, 'volume': 140869088, 'datetime': 1578636000000}, {'open': 77.91, 'high': 79.2675, 'low': 77.7875, 'close': 79.24, 'volume': 122086888, 'datetime': 1578895200000}, {'open': 79.175, 'high': 79.3925, 'low': 78.0425, 'close': 78.17, 'volume': 162613828, 'datetime': 1578981600000}, {'open': 77.9625, 'high': 78.875, 'low': 77.3875, 'close': 77.835, 'volume': 121923528, 'datetime': 1579068000000}, {'open': 78.3975, 'high': 78.925, 'low': 78.0225, 'close': 78.81, 'volume': 108829016, 'datetime': 1579154400000}, {'open': 79.0675, 'high': 79.685, 'low': 78.75, 'close': 79.6825, 'volume': 137816468, 'datetime': 1579240800000}, {'open': 79.2975, 'high': 79.755, 'low': 79.0, 'close': 79.1425, 'volume': 110843256, 'datetime': 1579586400000}, {'open': 79.645, 'high': 79.9975, 'low': 79.3275, 'close': 79.425, 'volume': 101832460, 'datetime': 1579672800000}, {'open': 79.48, 'high': 79.89, 'low': 78.9125, 'close': 79.8075, 'volume': 104471972, 'datetime': 1579759200000}, {'open': 80.0625, 'high': 80.8325, 'low': 79.3797, 'close': 79.5775, 'volume': 146537520, 'datetime': 1579845600000}, {'open': 77.515, 'high': 77.9425, 'low': 76.22, 'close': 77.2375, 'volume': 161940020, 'datetime': 1580104800000}, {'open': 78.15, 'high': 79.6, 'low': 78.0475, 'close': 79.4225, 'volume': 162233944, 'datetime': 1580191200000}, {'open': 81.1125, 'high': 81.9625, 'low': 80.345, 'close': 81.085, 'volume': 216599712, 'datetime': 1580277600000}, {'open': 80.135875, 'high': 81.0225, 'low': 79.6875, 'close': 80.9675, 'volume': 126743232, 'datetime': 1580364000000}, {'open': 80.2325, 'high': 80.67, 'low': 77.0725, 'close': 77.3775, 'volume': 199588384, 'datetime': 1580450400000}]

In [ ]:
ticker.info()

In [ ]:
df = pd.DataFrame(candles)
#    df["price"] = df.apply(lambda x: get_adjusted_close(x["symbol"], trading_day), axis=1)
# apply isn't ideal here, as it is essentially iterating through all rows, but I don't know of a better alternative
# for the size of this dataset I think this is a reasonable hit to take on performance
df['Date'] = df.apply(lambda x: arrow.get(x['datetime']).date(), axis=1)
df["Date"] = pd.to_datetime(df["Date"])
#df["Date"] = pd.to_datetime(arrow.get(df["datetime"]))
# Using Quandl's format as inspiration for how this dataframe _should_ look like
# Moving from raw api response to a dataframe will be a best opportunity to transform
# data from numerous APIs into a canonical format, which for now is just like Quandl :)
# new_column_names={'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close', 'Volume': 'volume'}
new_column_names={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}
# df
df = df.drop(['datetime'], axis=1).set_index('Date').rename(columns=new_column_names)
df.info()

In [ ]:
df.to_csv('../tests/fixtures/get_daily_price_history_aapl.csv')

In [ ]:
print(df.info())

In [ ]:
ticker['daily_returns'] = ticker['AdjClose'].pct_change()

In [ ]:
monthly_returns = ticker['daily_returns'].resample('M').apply(lambda x: ((x + 1).cumprod()).last("D"))

In [ ]:
monthly_returns

In [ ]:
twelve_month_lookback_momentum = np.prod(monthly_returns) - 1
print(twelve_month_lookback_momentum)